In [ ]:
!pip install numpy pathlib pandas gdown lxml hf_transfer scikit-learn tqdm peft==0.13.2 transformers==4.46.3

In [2]:
!gdown --folder "https://drive.google.com/drive/folders/1wAS0umYohuR53r4sqroxxiG2ab5p5msn"

^C


In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from peft import LoraConfig, get_peft_model

from transformers.models.esm.tokenization_esm import EsmTokenizer

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

import pandas as pd

import os
import random
import numpy as np

c:\Users\PNC\anaconda3\envs\GLM_311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\PNC\anaconda3\envs\GLM_311\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\PNC\anacon

In [4]:
def set_seed(seed: int=7) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(7)

In [5]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

DATA_PATH = "./genomic_language_model/"
df = pd.read_csv(DATA_PATH + "fine_tuning.csv")

ref_seq = df["reference_seq"]
var_seq = df["variant_seq"]
label = df["label"]

train_df, val_df = train_test_split(df, test_size=0.1, shuffle=True, stratify=df["label"])

In [6]:
max_seq_len = max(ref_seq.str.len().max(), var_seq.str.len().max())
print(f"Rows = {len(df):,}, Max Sequence Length = {max_seq_len}")

Rows = 397,182, Max Sequence Length = 512


In [7]:
BATCH_SIZE = 16

MODEL_ID = "InstaDeepAI/nucleotide-transformer-v2-500m-multi-species"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
backbone = AutoModelForMaskedLM.from_pretrained(MODEL_ID, trust_remote_code=True)

MODEL_CAP = tokenizer.model_max_length
MAX_LEN = min(MODEL_CAP, max_seq_len)

A new version of the following files was downloaded from https://huggingface.co/InstaDeepAI/nucleotide-transformer-v2-500m-multi-species:
- esm_config.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/InstaDeepAI/nucleotide-transformer-v2-500m-multi-species:
- modeling_esm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Processing file 1dw1yoa2ZbdCBNOY4rPNKwS2fZfUR_O3F fine_tuning.csv
Processing file 1lrRuZCk4xERL8CbeD2c5B_0v0fNY00o_ sample_submission.csv
Processing file 1nZ9YFe5dVR_Ipq-LpD8hmtZ9OnODcyz- test.csv


Retrieving folder contents
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1dw1yoa2ZbdCBNOY4rPNKwS2fZfUR_O3F
From (redirected): https://drive.google.com/uc?id=1dw1yoa2ZbdCBNOY4rPNKwS2fZfUR_O3F&confirm=t&uuid=84fe37eb-029e-44dc-ad03-ceabb291fe29
To: f:\AI-Contest\Genomic_Language_Model\genomic_language_model\fine_tuning.csv

  0%|          | 0.00/409M [00:00<?, ?B/s]
  0%|          | 524k/409M [00:00<02:30, 2.71MB/s]
  1%|          | 2.10M/409M [00:00<00:58, 6.98MB/s]
  1%|          | 3.67M/409M [00:00<00:45, 8.95MB/s]
  1%|▏         | 5.24M/409M [00:00<00:40, 9.99MB/s]
  2%|▏         | 6.82M/409M [00:00<00:37, 10.6MB/s]
  2%|▏         | 8.39M/409M [00:00<00:36, 11.0MB/s]
  2%|▏         | 9.96M/409M [00:01<00:35, 11.3MB/s]
  3%|▎         | 11.5M/409M [00:01<00:34, 11.4MB/s]
  3%|▎         | 13.1M/409M [00:01<00:34, 11.6MB/s]
  4%|▎         | 14.7M/409M [00:01<00:33, 11

In [8]:
backbone

EsmForMaskedLM(
  (esm): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(4107, 1024, padding_idx=1)
      (dropout): Dropout(p=0.0, inplace=False)
      (position_embeddings): Embedding(2050, 1024, padding_idx=1)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-28): 29 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (LayerNorm): LayerNorm((1024,), eps=1e-12

In [9]:
class SiameseDataset(Dataset):
    def __init__(self, df: pd.DataFrame, tokenizer: EsmTokenizer, max_len: int) -> None:
        self.ref_seq = df["reference_seq"].tolist()
        self.var_seq = df["variant_seq"].tolist()
        self.label = df["label"].tolist()
        self.tok = tokenizer
        self.max_len = max_len

    def __len__(self) -> int:
        return len(self.ref_seq)
    
    def __getitem__(self, idx: int) -> dict:
        return {"ref_seq": self.ref_seq[idx], "var_seq" : self.var_seq[idx], "label" : torch.tensor(self.label[idx], dtype=torch.float)}
    
def collate_fn(batch: torch.tensor, tok: EsmTokenizer=tokenizer, max_len: int=MAX_LEN) -> dict:
    ref_seq = [b["ref_seq"] for b in batch]
    var_seq = [b["var_seq"] for b in batch]
    label = torch.stack([b["label"] for b in batch])
    
    ref_enc = tok.batch_encode_plus(
        ref_seq,
        return_tensors="pt",
        padding="longest",
        truncation=True,
        max_length=max_len
    )

    var_enc = tok.batch_encode_plus(
        var_seq,
        return_tensors="pt",
        padding="longest",
        truncation=True,
        max_length=max_len
    )

    return {
        "ref_input_ids": ref_enc["input_ids"],
        "ref_attention_mask": ref_enc["attention_mask"],
        "var_input_ids": var_enc["input_ids"],
        "var_attention_mask": var_enc["attention_mask"],
        "label": label
    }

In [10]:
train_dataset = SiameseDataset(train_df, tokenizer, MAX_LEN)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                    collate_fn=collate_fn)

val_dataset = SiameseDataset(val_df, tokenizer, MAX_LEN)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False,
                         collate_fn=collate_fn)

In [11]:
class BackboneModel(nn.Module):
    def __init__(self, backbone, reconstruction_dim: int=2048) -> None:
        super().__init__()
        self.backbone = backbone
        
        for name, p in self.backbone.named_parameters():
            if "lora_" not in name:
                p.requires_grad = False

        hidden_size = backbone.config.hidden_size
        self.reconstruction_layer = nn.Sequential(
            nn.Linear(hidden_size, hidden_size * 2),
            nn.LayerNorm(hidden_size * 2),
            nn.GELU(),
            nn.Linear(hidden_size * 2, reconstruction_dim)
        )

    def forward(self, input_ids: torch.tensor, attention_mask: torch.tensor) -> torch.tensor:
        outs = self.backbone(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )

        last_hidden = outs.hidden_states[-1]
        mask_exp = attention_mask.unsqueeze(-1)

        summed = (last_hidden * mask_exp).sum(dim=1)
        counts = mask_exp.sum(dim=1).clamp(min=1)
        seq_emb = summed / counts

        seq_emb = self.reconstruction_layer(seq_emb)
        
        return seq_emb

In [12]:
class SiameseModel(nn.Module):
    def __init__(self, backbone, reconstruction_dim: int=2048) -> None:
        super().__init__()
        self.encoder = BackboneModel(backbone, reconstruction_dim)

    def forward(self, ref_input_ids, ref_attention_mask,
                      var_input_ids, var_attention_mask):
        ref_emb = self.encoder(ref_input_ids, ref_attention_mask)
        var_emb = self.encoder(var_input_ids, var_attention_mask)

        return ref_emb, var_emb

In [ ]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin: float=1.0):
        super().__init__()
        self.margin = margin

    def forward(self, emb1, emb2, label):
        emb1 = F.normalize(emb1, dim=-1)
        emb2 = F.normalize(emb2, dim=-1)
        
        dist = 1 - F.cosine_similarity(emb1, emb2, dim=-1)
        loss = label * dist + (1 - label) * F.relu(self.margin - dist)

        return 30 * loss.mean()

In [14]:
def validating(model, valDL, data_size, loss_fn, device):
    model.eval()

    loss_total = 0

    use_amp = (DEVICE == "cuda")

    with torch.no_grad():
        for batch in valDL:
            ref_ids = batch["ref_input_ids"].to(device)
            ref_attn_mask = batch["ref_attention_mask"].to(device)
            var_ids = batch["var_input_ids"].to(device)
            var_attn_mask = batch["var_attention_mask"].to(device)
            label = batch["label"].to(device)

            batch_size = len(ref_ids)
            
            with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=use_amp):
                ref_emb, var_emb = model(ref_ids, ref_attn_mask, var_ids, var_attn_mask)
                loss = loss_fn(ref_emb, var_emb, label).to(device)

            loss_total += loss.item() * batch_size
    
    avg_loss = loss_total / data_size

    return avg_loss


In [15]:
def training(model, trainDL, valDL, optimizer, epoch,
            data_size, val_data_size, loss_fn,
            scheduler, device, lora):
    SAVE_PATH = "./saved_models"
    os.makedirs(SAVE_PATH, exist_ok=True)

    BREAK_CNT_LOSS = 0
    LIMIT_VALUE = 5

    LOSS_HISTORY = [[], []]

    use_amp = (DEVICE == "cuda")

    for count in range(1, epoch + 1):
        model.train()

        SAVE_WEIGHT = os.path.join(SAVE_PATH, f'model_weights.pth')
        SAVE_LORA_WEIGHT = os.path.join(SAVE_PATH, f"best_lora_weights")

        loss_total = 0

        for batch in trainDL:
            ref_ids = batch["ref_input_ids"].to(device)
            ref_attn_mask = batch["ref_attention_mask"].to(device)
            var_ids = batch["var_input_ids"].to(device)
            var_attn_mask = batch["var_attention_mask"].to(device)
            label = batch["label"].to(device)

            batch_size = len(ref_ids)

            with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=use_amp):
                ref_emb, var_emb = model(ref_ids, ref_attn_mask, var_ids, var_attn_mask)
                loss = loss_fn(ref_emb, var_emb, label).to(device)

            loss_total += loss.item() * batch_size

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        val_loss = validating(model, valDL, val_data_size, loss_fn, device)

        LOSS_HISTORY[0].append(loss_total / data_size)
        LOSS_HISTORY[1].append(val_loss)

        print(f"[{count} / {epoch}]\n - TRAIN LOSS : {LOSS_HISTORY[0][-1]}")
        print(f"VAL LOSS : {LOSS_HISTORY[1][-1]}")

        scheduler.step(val_loss)

        if len(LOSS_HISTORY[0]) >= 2:
            if LOSS_HISTORY[0][-1] >= LOSS_HISTORY[0][-2]: BREAK_CNT_LOSS += 1
        
        if len(LOSS_HISTORY[0]) == 1:
            lora.save_pretrained(SAVE_LORA_WEIGHT)
            torch.save(model.encoder.reconstruction_layer.state_dict(), SAVE_WEIGHT)
        
        else:
            if LOSS_HISTORY[0][-1] < min(LOSS_HISTORY[0][:-1]):
                lora.save_pretrained(SAVE_LORA_WEIGHT)
                torch.save(model.encoder.reconstruction_layer.state_dict(), SAVE_WEIGHT)
        
        if BREAK_CNT_LOSS > LIMIT_VALUE:
            print(f"성능 및 손실 개선이 없어서 {count} EPOCH에 학습 중단")
            break
    
    return LOSS_HISTORY
    

In [16]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query", "key", "value"],
    lora_dropout=0,
    bias="none",
    task_type="CAUSAL_LM"
)

lora = get_peft_model(backbone, lora_config)

In [17]:
LR = 1e-4
EPOCH = 100
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

model = SiameseModel(lora).to(DEVICE)

params = [p for n, p in model.named_parameters() if p.requires_grad]
optimizer = optim.AdamW(params, lr=LR)

loss_fn = ContrastiveLoss()
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=3)

data_size = len(train_dataset)
val_data_size = len(val_dataset)

In [18]:
model.encoder.backbone.print_trainable_parameters()

trainable params: 2,850,816 || all params: 501,196,252 || trainable%: 0.5688


In [ ]:
loss = training(model=model, trainDL=train_loader, valDL=val_loader, optimizer=optimizer,
                epoch=EPOCH, data_size=data_size, val_data_size=val_data_size,
                loss_fn=loss_fn, scheduler=scheduler, device=DEVICE, lora=lora)

OutOfMemoryError: CUDA out of memory. Tried to allocate 12.00 MiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Of the allocated memory 14.40 GiB is allocated by PyTorch, and 203.13 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [1]:
data_path = './genomic_language_model/'
df = pd.read_csv(data_path + 'test.csv')

max_seq_len = df["seq"].str.len().max()
EFFECTIVE_MAX_LEN = min(MODEL_CAP, max_seq_len)

EFFECTIVE_MAX_LEN

NameError: name 'pd' is not defined

In [ ]:
class SeqDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.ids  = df["ID"].tolist()
        self.seqs = df["seq"].tolist()
        self.tok  = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        return {"ID": self.ids[idx], "seq": self.seqs[idx]}

def collate_fn(batch, tok=tokenizer, max_len=EFFECTIVE_MAX_LEN):
    ids  = [b["ID"] for b in batch]
    seqs = [b["seq"] for b in batch]
    enc  = tok.batch_encode_plus(
        seqs,
        return_tensors="pt",
        padding="longest",          
        truncation=True,
        max_length=max_len
    )
    # attention_mask: pad 토큰이 0
    return {
        "ids": ids,
        "input_ids": enc["input_ids"],
        "attention_mask": enc["attention_mask"]
    }

In [ ]:
dataset = SeqDataset(df, tokenizer, EFFECTIVE_MAX_LEN)
loader  = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False,
                     collate_fn=collate_fn)
print("✅ Dataloader ready.")

In [ ]:
lora = PeftModel.from_pretrained(
    backbone,
    "./saved_models/best_lora_weights"
)

lora_model = BackboneModel(lora).to(DEVICE)
lora_model.reconstruction_layer.load_state_dict(torch.load("./saved_models/model_weights.pth", weights_only=True))

all_ids = []
all_embs = []
use_amp = (DEVICE == "cuda")

lora_model.eval()
with torch.no_grad():
    for batch in loader:
        input_ids = batch["input_ids"].to(DEVICE)
        attn_mask = batch["attention_mask"].to(DEVICE)

        with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=use_amp):
            outs = lora_model(
                input_ids,
                attention_mask=attn_mask
            )
    
        all_ids.extend(batch["ids"])
        all_embs.append(outs.detach().cpu())

emb = torch.vstack(all_embs).float()        # (N, H)
N, H = emb.shape
print(f"✅ Embedding shape = {N} x {H}")

In [ ]:
sample_submission = pd.read_csv(data_path + 'sample_submission.csv')

emb_np = emb.numpy()
emb_cols = [f"emb_{i:04d}" for i in range(emb_np.shape[1])]
emb_df = pd.DataFrame(emb_np, columns=emb_cols)

submission = pd.concat([sample_submission['ID'], emb_df], axis=1)
submission.to_csv('lora_submission.csv', index=False)